#### Braket

In [ ]:
from braket import circuits as bk_circuits
from braket import devices as bk_devices

In [ ]:
bk_circuit = bk_circuits.Circuit()

theta = [np.pi/4, np.pi/2, 3*np.pi/4, np.pi]

bk_circuit.h([0, 1])
bk_circuit.ry(0, theta[0])
bk_circuit.rz(1, theta[1])
bk_circuit.ry(0, theta[2])
bk_circuit.rz(1, theta[3])

print(bk_circuit)

In [ ]:
# bk_sim = bk_devices.LocalSimulator(backend="braket_sv")
# bk_sim_job = bk_sim.run(bk_circuit, shots=1024)
# bk_sim_result = bk_sim_job.result()
# bk_sim_result.measurement_counts

# qBraid-SDK

### Search for find all quantum devices available through Qiskit, Cirq, and Braket

There's no such function in Qiskit or Cirq, only tedious work-arounds that require you to look into the docs anyways. For Braket, it requires making an authenticated API request through Boto3, and only includes credentialed devices in the response.

The qBraid-SDK lists all available QPU's and simulators available through Qiskit, Cirq, and Braket, all in one place.

In [1]:
from qbraid import get_devices, device_wrapper

In [2]:
get_devices()

Provider,Name,qBraid ID,Status
AWS,Braket Default Simulator,aws_braket_default_sim,●
AWS,Density Matrix Simulator,aws_dm_sim,●
AWS,State Vector Simulator,aws_sv_sim,●
AWS,Tensor Network Simulator,aws_tn_sim,●
D-Wave,Advantage_system4,aws_dwave_advantage_system4,●
D-Wave,DW_2000Q_6,aws_dwave_2000Q_6,●
Google,Bristlecone,google_bristlecone,○
Google,Cirq Density Matrix Simulator,google_cirq_dm_sim,●
Google,Cirq Sparse Simulator,google_cirq_sparse_sim,●
Google,Foxtail,google_foxtail,○


Filter your device search with many different advanced query options! 

For example, let's find simulators containing keyword "State" available through AWS or IBM:

In [4]:
get_devices({"type": "Simulator", "name": {"$regex": "State"}, "vendor": {"$in": ["AWS", "IBM"]}})

Provider,Name,qBraid ID,Status
AWS,State Vector Simulator,aws_sv_sim,●
IBM,Aer State Vector Simulator,ibm_aer_sv_sim,●
IBM,BasicAer State Vector Simulator,ibm_basicaer_sv_sim,●
IBM,IBMQ Matrix Product State Simulator,ibm_q_mps_sim,●
IBM,IBMQ State Vector Simulator,ibm_q_sv_sim,●


This time, let's search for all gate-based QPUs with at least 5 qubits that are online

In [5]:
get_devices({"paradigm": "gate-based", "type": "QPU", "qubits": {"$gte": 5}, "status": "ONLINE"})

Provider,Name,qBraid ID,Status
IBM,IBMQ Belem,ibm_q_belem,●
IBM,IBMQ Lima,ibm_q_lima,●
IBM,IBMQ Quito,ibm_q_quito,●
IBM,IBMQ Santiago,ibm_q_manila,●
IonQ,IonQ Device,aws_ionq,●
Rigetti,Aspen-11,aws_rigetti_aspen_11,●


Not sure how to construct your query? 

In [7]:
help(get_devices)

Help on function get_devices in module qbraid.devices._utils.device_api:

get_devices(query=None)
    Displays a list of all supported devices matching given filters, tabulated by provider,
    name, and qBraid ID. Each device also has a status given by a solid green bubble or a hollow
    red bubble, indicating that the device is online or offline, respectively. You can narrow your
    device search by supplying a dictionary containing the desired criteria. Available filters
    include:
    
    * name (str)
    * vendor (str): AWS | IBM | Google
    * provider (str): AWS | IBM | Google | D-Wave | IonQ | Rigetti
    * type (str): QPU | Simulator
    * qubits (int)
    * paradigm (str): gate-based | quantum-annealer
    * requires_cred (bool): true | false
    * status (str): ONLINE | OFFLINE
    
    Here are a few example use cases:
    
    .. code-block:: python
    
        from qbraid import get_devices
    
        # Search for gate-based devices provided by Google that are onl

Recently added the ability to search by preferred software package, for instance

In [16]:
get_devices({"run_package": "qiskit", "requires_cred": "false"})

Provider,Name,qBraid ID,Status
IBM,Aer Default Simulator,ibm_aer_default_sim,●
IBM,Aer Pulse Simulator,ibm_aer_pulse_sim,●
IBM,Aer QASM Simulator,ibm_aer_qasm_sim,●
IBM,Aer State Vector Simulator,ibm_aer_sv_sim,●
IBM,Aer Unitary Simulator,ibm_aer_unitary_sim,●
IBM,BasicAer QASM Simulator,ibm_basicaer_qasm_sim,●
IBM,BasicAer State Vector Simulator,ibm_basicaer_sv_sim,●
IBM,BasicAer Unitary Simulator,ibm_basicaer_unitary_sim,●


### Apply the qbraid device wrapper and run your circuit

The qbraid device wrapper has an array of useful features. Quickly collect information about the device, or extract the wrapped device object

In [44]:
qbraid_device = device_wrapper("ibm_aer_qasm_sim")
qbraid_device.info

{'qbraid_id': 'ibm_aer_qasm_sim',
 'name': 'Aer QASM Simulator',
 'provider': 'IBM',
 'paradigm': 'gate-based',
 'type': 'Simulator',
 'vendor': 'IBM',
 'run_package': 'qiskit',
 'status': 'ONLINE',
 'qubits': None}

In [45]:
qiskit_backend = qbraid_device.vendor_dlo  # vendor device-like-object
type(qiskit_backend)

qiskit.providers.aer.backends.qasm_simulator.QasmSimulator

In [46]:
from qiskit import QuantumCircuit
import numpy as np

qiskit_circuit = QuantumCircuit(2)

theta = [np.pi/4, np.pi/2, 3*np.pi/4, np.pi]

qiskit_circuit.h([0, 1])
qiskit_circuit.ry(theta[0], 0)
qiskit_circuit.rz(theta[1], 1)
qiskit_circuit.ry(theta[2], 0)
qiskit_circuit.rz(theta[3], 1)

qiskit_circuit.draw()

┌───┐┌─────────┐┌──────────┐
q_0: ┤ H ├┤ Ry(π/4) ├┤ Ry(3π/4) ├
     ├───┤├─────────┤└┬───────┬─┘
q_1: ┤ H ├┤ Rz(π/2) ├─┤ Rz(π) ├──
     └───┘└─────────┘ └───────┘

Initialize quantum jobs and collect results, with the ability to extract wrapped objects at any step

In [50]:
qbraid_job = qbraid_device.run(qiskit_circuit)
qbraid_job.vendor_jlo  # vendor job-like-object

In [51]:
qbraid_result = qbraid_job.result()
qbraid_result.vendor_rlo  # vendor result-like-object

Result(backend_name='qasm_simulator', backend_version='0.10.2', qobj_id='a30193f6-221c-4a91-bbd9-a4edfda2bf53', job_id='6972afe7-04be-4bc3-994c-bf5c0f475a72', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(clbit_labels=[], creg_sizes=[], global_phase=0.7853981633974483, memory_slots=0, metadata=None, n_qubits=2, name='circuit-547', qreg_sizes=[['q', 2]], qubit_labels=[['q', 0], ['q', 1]]), status=DONE, seed_simulator=3503567375, metadata={'parallel_state_update': 8, 'batched_shots_optimization': False, 'measure_sampling': False, 'device': 'CPU', 'num_qubits': 0, 'parallel_shots': 1, 'remapped_qubits': False, 'method': 'stabilizer', 'active_input_qubits': [], 'num_clbits': 0, 'input_qubit_map': []}, time_taken=1.1005e-05)], date=2022-01-29T15:43:13.579409, status=COMPLETED, metadata={'time_taken': 0.000137199, 'time_taken_execute': 3.9128e-05, 'parallel_experiments': 1, 'omp_enabled': True, 'max_gp

### Run your circuit on any device and compare results

In [53]:
qbraid_google_device = device_wrapper("google_cirq_dm_sim")

In [56]:
qbraid_aws_device = device_wrapper("aws_dm_sim")  # requires aws credential

AWS Access Key ID [None]:  ····················


KeyboardInterrupt: Interrupted by user

In [ ]:
bk_unitary = bk_circuits.unitary_calculation.calculate_unitary(bk_circuit.qubit_count, bk_circuit.instructions)

In [ ]:
bk_sim_result = bk_dm.run(bk_circuit, shots=1024).result()

In [ ]:
bk_sim_result.measurements
# bk_sim_result.measurement_counts
# bk_sim_result.measurement_probabilities

#### Cirq

In [ ]:
import cirq

In [ ]:
print([cls.__name__ for cls in cirq.Device.__subclasses__()])

In [ ]:
cirq_sim = cirq.DensityMatrixSimulator()

In [ ]:
qs = cirq.LineQubit.range(2)
# qs.reverse()

cirq_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.H(q1),
    cirq.ry(rads=theta[0])(qs[0]),
    cirq.rz(rads=theta[1])(qs[1]),
    cirq.ry(rads=theta[2])(qs[0]),
    cirq.rz(rads=theta[3])(qs[1]),
    cirq.measure(q0, key=str(qs[0].x)),
    cirq.measure(q1, key=str(qs[1].x)),
)

print(cirq_circuit)

In [ ]:
cirq_unitary = cirq_circuit.unitary()

In [ ]:
cirq_sim_result = cirq_sim.run(cirq_circuit, repetitions=1024)

In [ ]:
cirq_sim_result.measurements
# cirq_sim_result.data
# cirq_sim_result.histogram
# cirq_sim_result.multi_measurement_histogram(keys=cirq_sim_result.measurements.keys())

In [ ]:
cirq_sim_meas = cirq_sim_result.measurements

In [ ]:
print(cirq_sim_meas)

In [ ]:
from braket.circuits import Circuit as BkCircuit
from braket.devices import LocalSimulator as BkSimulator
from braket.aws import AwsDevice

import numpy as np

In [ ]:
aws_dm1 = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

In [ ]:
# Please enter the S3 bucket you created during onboarding in the code below

my_bucket = os.environ["s3_bucket"]
my_prefix = os.environ["s3_prefix"]

# my_bucket = "amazon-braket-Your-Bucket-Name" # the name of the bucket
# my_prefix = "Your-Folder-Name" # the name of the folder in the bucket

s3_folder = (my_bucket, my_prefix)

In [ ]:
aws_dm1_job = aws_dm_sim.run(bk_circuit, s3_folder, shots=1024)

In [ ]:
aws_dm1_job.metadata()["status" ]

In [ ]:
aws_dm1_result = job.result()

In [ ]:
aws_dm1_counts = aws_dm1_result.measurement_counts

In [ ]:
print(aws_dm1_counts) # <class 'collections.Counter'>

In [ ]:
import qbraid

In [ ]:
aws_device = device_wrapper("aws_dm_sim")

In [ ]:
aws_device._s3_location

In [ ]:
import os
 = 'amazon-braket-592242689881'
 = 'simulator-output'

In [ ]:
from qbraid import (
    get_devices, 
    refresh_devices, 
    random_circuit, 
    device_wrapper, 
)

Get a complete list of devices available through qBraid using the get_devices function

In [ ]:
get_devices()

In [ ]:
get_devices({"paradigm": "gate-based", "type": "QPU", "qubits": {"$gte": 5}, "status": "ONLINE"})

This time, let's search for state vector simulators available through AWS or IBM

In [ ]:
get_devices({"type": "Simulator", "name": {"$regex": "Density Matrix"}, "vendor": {"$in": ["AWS", "Google"]}})

The qBraid SDK makes running quantum circuits on any device extremely easy. 

Simply copy the desired qBraid ID into the device wrapper function and your on your way

In [ ]:
aws_device = device_wrapper("aws_dm_sim")

In [ ]:
google_device = device_wrapper("google_cirq_dm_sim")

In [ ]:
ibm_device = device_wrapper("ibm_q_lima") 

In [ ]:
ibm_device.info

In [ ]:
circuit = random_circuit("qiskit", num_qubits=3, depth=5, measure=True)
print(circuit)

In [ ]:
shots=1000

In [ ]:
aws_job = aws_device.run(circuit, shots=shots)

In [ ]:
google_job = google_device.run(circuit, shots=shots)

In [ ]:
ibm_job = ibm_device.run(circuit, shots=shots)

In [ ]:
aws_job.id

In [ ]:
# ibm_job.device.pending_jobs()
ibm_job.status()

In [ ]:
ibm_result = ibm_job.result()

In [ ]:
google_result = google_job.result()

In [ ]:
aws_result = aws_job.result()

In [ ]:
aws_result.measurement_counts()

In [ ]:
google_result.measurement_counts()

In [ ]:
ibm_result.measurement_counts()

In [ ]:
ibm_result.plot_counts()

In [ ]:
aws_result.plot_counts()

In [ ]:
google_result.plot_counts()